In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt

In [3]:
import pathlib

In [4]:
import xarray as xr
import dask
from dask.diagnostics import ProgressBar

### parameters for papermill 

In [5]:
varname = 'Rh'
first_year = 1972
last_year = 2021

In [6]:
dpath_vcsn = f"/media/nicolasf/END19101/data/VCSN/daily/{varname}"

In [7]:
dpath_vcsn = pathlib.Path(dpath_vcsn)

In [8]:
lfiles = list(dpath_vcsn.glob("*.nc"))

In [9]:
lfiles.sort()

In [10]:
lfiles

[PosixPath('/media/nicolasf/END19101/data/VCSN/daily/Rh/VCSN_daily_Rh_1972-01.nc'),
 PosixPath('/media/nicolasf/END19101/data/VCSN/daily/Rh/VCSN_daily_Rh_1972-02.nc'),
 PosixPath('/media/nicolasf/END19101/data/VCSN/daily/Rh/VCSN_daily_Rh_1972-03.nc'),
 PosixPath('/media/nicolasf/END19101/data/VCSN/daily/Rh/VCSN_daily_Rh_1972-04.nc'),
 PosixPath('/media/nicolasf/END19101/data/VCSN/daily/Rh/VCSN_daily_Rh_1972-05.nc'),
 PosixPath('/media/nicolasf/END19101/data/VCSN/daily/Rh/VCSN_daily_Rh_1972-06.nc'),
 PosixPath('/media/nicolasf/END19101/data/VCSN/daily/Rh/VCSN_daily_Rh_1972-07.nc'),
 PosixPath('/media/nicolasf/END19101/data/VCSN/daily/Rh/VCSN_daily_Rh_1972-08.nc'),
 PosixPath('/media/nicolasf/END19101/data/VCSN/daily/Rh/VCSN_daily_Rh_1972-09.nc'),
 PosixPath('/media/nicolasf/END19101/data/VCSN/daily/Rh/VCSN_daily_Rh_1972-10.nc'),
 PosixPath('/media/nicolasf/END19101/data/VCSN/daily/Rh/VCSN_daily_Rh_1972-11.nc'),
 PosixPath('/media/nicolasf/END19101/data/VCSN/daily/Rh/VCSN_daily_Rh_1972-1

In [11]:
vcsn_dset = xr.open_mfdataset(lfiles, concat_dim='time', combine='nested')

In [12]:
vcsn_dset

<xarray.Dataset>
Dimensions:  (time: 18263, lat: 257, lon: 241)
Coordinates:
  * time     (time) datetime64[ns] 1972-01-01 1972-01-02 ... 2021-12-31
  * lat      (lat) float64 -47.27 -47.23 -47.17 -47.12 ... -34.52 -34.48 -34.42
  * lon      (lon) float64 166.5 166.5 166.6 166.6 ... 178.3 178.4 178.4 178.5
Data variables:
    Rh       (time, lat, lon) float64 dask.array<chunksize=(31, 257, 241), meta=np.ndarray>

In [13]:
vcsn_dset = vcsn_dset.chunk({'time':-1, 'lat':10, 'lon':10})

In [14]:
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    vcsn_dset = vcsn_dset.resample({'time':'1M'}).mean('time')

In [15]:
list(map(str, [first_year, last_year]))

['1972', '2021']

In [16]:
vcsn_dset = vcsn_dset.sel(time=slice(*list(map(str, [first_year, last_year]))))

In [17]:
vcsn_dset

,Array,Chunk
Bytes,283.53 MiB,468.75 kiB
Shape,"(600, 257, 241)","(600, 10, 10)"
Count,9347 Tasks,650 Chunks
Type,float64,numpy.ndarray


In [18]:
with ProgressBar(): 
    vcsn_dset.to_netcdf(f'/media/nicolasf/END19101/data/PICT/datasets/VCSN/VCSN_monthly_{varname}_{first_year}_{last_year}_grid.nc')

[#####################                   ] | 54% Completed |  6.0s

/home/nicolasf/mambaforge/envs/climetlab/lib/python3.8/site-packages/flox/aggregate_flox.py:105: RuntimeWarning: invalid value encountered in true_divide
  out /= nanlen(group_idx, array, size=size, axis=axis, fill_value=0)


[########################################] | 100% Completed | 23.1s


In [19]:
varname

'Rh'